# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
R. Andrassy  ->  R. Andrassy  |  ['R. Andrassy']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
C. Larkin  ->  C. Larkin  |  ['C. Larkin']
U. Chadayammuri  ->  U. Chadayammuri  |  ['U. Chadayammuri']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
Arxiv has 49 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.02556


extracting tarball to tmp_2407.02556...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


...

✔ → 19944:\section{A zoo of modelled close-in giants}\label{sec:full-sample}
  ↳ 29480:\section{Factors affecting planet compositions}\label{sec:assumption}


✔ → 29480:\section{Factors affecting planet compositions}\label{sec:assumption}
  ↳ 51305:\section{What might the BOWIE-ALIGN sample tell us about planet formation?}\label{sec:insight}
✔ → 51305:\section{What might the BOWIE-ALIGN sample tell us about planet formation?}\label{sec:insight}
  ↳ 55017:\section{Conclusions}\label{sec:conclusions}


✔ → 55017:\section{Conclusions}\label{sec:conclusions}
  ↳ 60183:\section{Planet model}


✘ → 60183:\section{Planet model}
  ↳ 79737:\section{Effects of free parameters} \label{sec:parameters}
✔ → 79737:\section{Effects of free parameters} \label{sec:parameters}
  ↳ 87296:end


Found 98 bibliographic references in tmp_2407.03199/paper.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.02556-b31b1b.svg)](https://arxiv.org/abs/2407.02556) | **Carbon and Iron Deficiencies in Quiescent Galaxies at z=1-3 from JWST-SUSPENSE: Implications for the Formation Histories of Massive Galaxies**  |
|| A. G. Beverage, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-07-05*|
|*Comments*| *Submitted to ApJ; 18 pages, 6 figures, 1 table*|
|**Abstract**|            We present the stellar metallicities and multi-element abundances (C, Mg, Si, Ca, Ti, Cr, and Fe) of 15 massive (log M/M$_\odot$=10.2-11.2) quiescent galaxies at z=1-3, derived from ultradeep JWST-SUSPENSE spectra. Compared to quiescent galaxies at z~0, these galaxies exhibit a deficiency of 0.25 dex in [C/H], 0.16 dex in [Fe/H], and 0.07 dex in [Mg/H], implying rapid formation and quenching before significant enrichment from asymptotic giant branch stars and Type Ia supernovae. Additionally, we find that galaxies that form at higher redshift have higher [Mg/Fe] and lower [Fe/H] and [Mg/H], irrespective of their observed redshift. The evolution in [Fe/H] and [C/H] is therefore primarily explained by lower redshift samples naturally including galaxies with longer star-formation timescales. On the other hand, the lower [Mg/H] can be explained by galaxies forming at earlier epochs expelling larger gas reservoirs during their quenching phase. Consequently, the mass-metallicity relation, primarily reflecting [Mg/H], is also lower at z=1-3 compared to the lower redshift relation, though the slopes are similar. Finally, we compare our results to standard stellar population modeling approaches employing solar abundance patterns and non-parametric star-formation histories (using Prospector). Our SSP-equivalent ages agree with the mass-weighted ages from Prospector, while the metallicities disagree significantly. Nonetheless, the metallicities better reflect [Fe/H] than total [Z/H]. We also find that star-formation timescales inferred from elemental abundances are significantly shorter than those from Prospector, and we discuss the resulting implications for the early formation of massive galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03137-b31b1b.svg)](https://arxiv.org/abs/2407.03137) | **X-Shooting ULLYSES: Massive Stars at low metallicity -- IV. Spectral analysis methods and exemplary results for O stars**  |
|| A. Sander, et al. -- incl., <mark>C. Larkin</mark> |
|*Appeared on*| *2024-07-05*|
|*Comments*| *18+15 pages, 21+4 figures, under review at A&A, condensed abstract*|
|**Abstract**|            CONTEXT: The spectral analysis of hot, massive stars is a fundamental astrophysical method to obtain their intrinsic properties and their feedback. Quantitative spectroscopy for hot, massive stars requires detailed numerical modeling of the atmosphere and an iterative treatment to obtain the best solution within a given framework. AIMS: We present an overview of different techniques for the quantitative spectroscopy of hot stars employed within the X-Shooting ULLYSES collaboration, from grid-based approaches to tailored fits. By performing a blind test, we gain an overview about the similarities and differences of the resulting parameters. Our study aims to provide an overview of the parameter spread caused by different approaches. METHODS: For three different stars from the sample (SMC O5 star AzV 377, LMC O7 star Sk -69 50, and LMC O9 star Sk -66 171), we employ different atmosphere codes (CMFGEN, Fastwind, PoWR) and strategies to determine their best-fitting model. For our analyses, UV and optical spectra are used to derive the properties with some methods relying purely on optical data for comparison. To determine the overall spectral energy distribution, we further employ additional photometry from the literature. RESULTS: Effective temperatures for each of three sample stars agree within 3 kK while the differences in log g can be up to 0.2 dex. Luminosity differences of up to 0.1 dex result from different reddening assumptions, which seem to be larger for the methods employing a genetic algorithm. All sample stars are nitrogen-enriched. CONCLUSIONS: We find a reasonable agreement between the different methods. Tailored fitting tends to be able to minimize discrepancies obtained with more course or automatized treatments. UV spectral data is essential for the determination of realistic wind parameters. For one target (Sk -69 50), we find clear indications of an evolved status.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03198-b31b1b.svg)](https://arxiv.org/abs/2407.03198) | **BOWIE-ALIGN: A JWST comparative survey of aligned vs misaligned hot Jupiters to test the dependence of atmospheric composition on migration history**  |
|| J. Kirk, et al. -- incl., <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2024-07-05*|
|*Comments*| *13 pages, 8 figures, submitted to RASTI*|
|**Abstract**|            A primary objective of exoplanet atmosphere characterisation is to learn about planet formation and evolution, however, this is challenged by degeneracies. To determine whether differences in atmospheric composition can be reliably traced to differences in evolution, we are undertaking a new survey with JWST to compare the compositions of a sample of hot Jupiters that orbit F stars above the Kraft break with different orbital alignments. Under the assumption that aligned planets migrate through the inner disc, while misaligned planets migrate after disc dispersal, the act of migrating through the inner disc should lead to a measurable difference in the C/O between aligned and misaligned planets. We expect the amplitude and sign of this difference to depend on the amount of planetesimal accretion and whether silicates accreted from the inner disc release their oxygen. Here, we identify all known exoplanets that are suitable for testing this hypothesis, describe our JWST survey, and use noise simulations and atmospheric retrievals to estimate our survey's sensitivity. With the selected sample of four aligned and four misaligned hot Jupiters, we will be sensitive to the predicted differences in C/O between aligned and misaligned hot Jupiters for a wide range of model scenarios.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03199-b31b1b.svg)](https://arxiv.org/abs/2407.03199) | **BOWIE-ALIGN: How formation and migration histories of giant planets impact atmospheric compositions**  |
|| A. B. Penzlin, et al. -- incl., <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2024-07-05*|
|*Comments*| *11pages 10 figures, (appendix: 6 page, 4 figures), submitted to mnras*|
|**Abstract**|            Hot Jupiters present a unique opportunity for measuring how planet formation history shapes present-day atmospheric composition. However, due to the myriad pathways influencing composition, a well-constructed sample of planets is needed to determine whether formation history can be accurately traced back from atmospheric composition. To this end, the BOWIE-ALIGN survey will compare the compositions of 8 hot Jupiters around F stars, 4 with orbits aligned with the stellar rotation axis and 4 misaligned. Using the alignment as an indicator for planets that underwent disc migration or high-eccentricity migration, one can determine whether migration history produces notable differences in composition between the two samples of planets. This paper describes the planet formation model that motivates our observing programme. Our model traces the accretion of chemical components from the gas and dust in the disc over a broad parameter space to create a full, unbiased model sample from which we can estimate the range of final atmospheric compositions. For high metallicity atmospheres (O/H > 10 times solar), the C/O ratios of aligned and misaligned planets diverge, with aligned planets having lower C/O (< 0.25) due to the accretion of oxygen-rich silicates from the inner disc. However, silicates may rain out instead of releasing their oxygen into the atmosphere. This would significantly increase the C/O of aligned planets (C/O > 0.6), inverting the trend between the aligned and misaligned planets. Nevertheless, by comparing statistically significant samples of aligned and misaligned planets, we expect atmospheric composition to constrain how planets form.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.02566-b31b1b.svg)](https://arxiv.org/abs/2407.02566) | **Large-scale ordered magnetic fields generated in mergers of helium white dwarfs**  |
|| R. Pakmor, et al. -- incl., <mark>R. Andrassy</mark> |
|*Appeared on*| *2024-07-05*|
|*Comments*| *12 pages, 9 figures, submitted to A&A, comments welcome*|
|**Abstract**|            Stellar mergers are one important path to highly magnetised stars. Mergers of two low-mass white dwarfs may create up to every third hot subdwarf star. The merging process is usually assumed to dramatically amplify magnetic fields. However, so far only four highly magnetised hot subdwarf stars have been found, suggesting a fraction of less than $1\%$. We present two high-resolution magnetohydrodynamical (MHD) simulations of the merger of two helium white dwarfs in a binary system with the same total mass of $0.6\,M_\odot$. We analyse one equal-mass merger with two $0.3\,M_\odot$ white dwarfs, and one unequal-mass merger with a $0.25\,M_\odot$ white dwarf and a $0.35\,M_\odot$ white dwarf. We simulate the inspiral, merger, and further evolution of the merger remnant for about $50$ rotations. We find efficient magnetic field amplification in both mergers via a small-scale dynamo, reproducing previous results of stellar merger simulations. The magnetic field saturates at similar strength for both simulations. We then identify a second phase of magnetic field amplification in both merger remnants that happens on a timescale of several tens of rotational periods of the merger remnant. This phase generates a large-scale ordered azimuthal field. We identify it as a large-scale dynamo driven by the magneto-rotational instability (MRI). Finally, we suggest that in the unequal-mass merger remnant, helium burning will eventually start in a shell around a cold core. The convection zone this generates will coincide with the region that contains most of the magnetic energy, probably erasing the strong, ordered field. The equal-mass merger remnant instead will probably ignite burning in the center, retaining its ordered field. Therefore, the mass ratio of the initial merger could be the selecting factor that decides if a merger remnant will stay highly magnetised long after the merger.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.03142-b31b1b.svg)](https://arxiv.org/abs/2407.03142) | **Closing Pandora's Box -- The deepest X-ray observations of Abell 2744 and a multi-wavelength merger picture**  |
|| <mark>U. Chadayammuri</mark>, Á. Bogdán, G. Schellenberger, J. ZuHone |
|*Appeared on*| *2024-07-05*|
|*Comments*| *Submitted to the Open Journal of Astrophysics*|
|**Abstract**|            Abell 2744, also known as Pandora's Cluster, is a complex merging galaxy cluster. While a major merger is clear along the north-south axis, the dynamical state of the northwest subcluster has been highly uncertain. We present ultra-deep ($\approx$2.1 Ms) X-ray observations of Abell 2744 obtained with the Chandra X-ray Observatory and reinterpret the multi-wavelength picture with a suite of idealised simulations of galaxy cluster mergers. The new data reveal in unprecedented detail the disruption of cool cores in the three X-ray luminous subclusters and confirm the presence of a shock to the NW. A position-velocity clustering of the cluster member galaxies shows a clearly separated S2 component, with a $\Delta z$ implying a separation of 53 Mpc or a line-of-sight velocity of $4500\ \rm{km \ s^{-1}}$, or likely some combination of the two. While binary simulations allow NW to have undergone a gravitational slingshot after the first pericenter passage, triple merger simulations rule out this scenario, because the two mergers would have had to occur $\sim$0.5 Gyr apart, and the joint impact of the shocks from the two mergers would completely disrupt the SE and NW cool cores; they only reform after 1-2 Gyr, by which point the core separations greatly exceed observations. The scenario that best describes Abell 2744 is a head-on N-S merger $0.5-0.6$ Gyrs ago followed by a first infall of the NW subcluster. Furthermore, we note that a model with three cluster-size halos, with masses consistent with gravitational lensing constraints, nevertheless produces a lensing convergence and surface brightness lower than observed in most of the field of view, whereas the temperatures are consistent with observations. This suggests the presence of a large-scale overdensity, which contributes to the diffuse emission and total surface density without heating the densest gas.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.02973-b31b1b.svg)](https://arxiv.org/abs/2407.02973) | **NOEMA formIng Cluster survEy (NICE): Characterizing eight massive galaxy groups at $1.5 < z < 4$ in the COSMOS field**  |
|| N. B. Sillassen, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-07-05*|
|*Comments*| *44 pages (27pp appendix), 32 figures, 18 tables, accepted for publication in A&A*|
|**Abstract**|            The NOEMA formIng Cluster survEy (NICE) is a large program targeting 69 massive galaxy group candidates at $z>2$ in six deep fields. We report spectroscopic confirmation of eight groups at $1.65\leq z\leq3.61$ in COSMOS. Homogeneously selected as significant overdensities of red IRAC sources with red Herschel colors, four groups are confirmed by CO and [CI] with NOEMA 3mm observations, three are confirmed with ALMA, and one is confirmed by H$\alpha$ from Subaru/FMOS. We constructed the integrated FIR SEDs for the eight groups, obtaining total IR SFR $=260-1300~{\rm M_\odot}$~yr$^{-1}$. We adopted six methods to estimate the dark matter masses, including stellar mass to halo mass relations, overdensity with galaxy bias, and NFW profile fitting to radial stellar mass density. We found the radial stellar mass density are consistent with a NFW profile, supporting that they are collapsed structures hosted by a single dark matter halo. The best halo mass estimates are $\log(M_{\rm h}/{\rm M_\odot})=12.8-13.7$ with uncertainty of 0.3 dex. From halo mass estimates, we derive baryonic accretion rate ${\rm BAR}=(1-8)\times10^{3}\,{\rm M_{\odot}/yr}$ for this sample. We find a quasi-linear correlation between the integrated SFR/BAR and the theoretical halo mass limit for cold streams, $M_{\rm stream}/M_{\rm h}$, with ${\rm SFR/BAR}=10^{-0.46\pm0.22}\left({M_{\rm stream}/M_{\rm h}}\right)^{0.71\pm0.16}$ with a scatter of $0.40\,{\rm dex}$. Further, we compare halo masses and stellar masses with simulations, and find all structures are consistent with being progenitors of $M_{\rm h}(z=0)>10^{14}\,{\rm M_{\odot}}$ galaxy clusters, and the most massive central galaxies have stellar masses consistent with brightest cluster galaxies (BCGs) progenitors in the TNG300 simulation. The results strongly suggest these structures are forming massive galaxy clusters via baryonic and dark matter accretion.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.02556/./fig4_abundances.png', 'tmp_2407.02556/./formation_time_v_xh_cM.png', 'tmp_2407.02556/./abundance_pattern_xfe_xh.png']
copying  tmp_2407.02556/./fig4_abundances.png to _build/html/
copying  tmp_2407.02556/./formation_time_v_xh_cM.png to _build/html/
copying  tmp_2407.02556/./abundance_pattern_xfe_xh.png to _build/html/
exported in  _build/html/2407.02556.md
    + _build/html/tmp_2407.02556/./fig4_abundances.png
    + _build/html/tmp_2407.02556/./formation_time_v_xh_cM.png
    + _build/html/tmp_2407.02556/./abundance_pattern_xfe_xh.png
found figures ['tmp_2407.03137/./Figures/uv2_AV377-crop.png', 'tmp_2407.03137/./Figures/SED_AV377_zoom-crop.png', 'tmp_2407.03137/./Figures/he-hydrogens_AV377-crop.png']
copying  tmp_2407.03137/./Figures/uv2_AV377-crop.png to _build/html/
copying  tmp_2407.03137/./Figures/SED_AV377_zoom-crop.png to _build/html/
copying  tmp_2407.03137/./Figures/he-hydrogens_AV377-crop.png to _build/html/
exported in  _build/html/2407.03137.

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\zsol}{\log\left(Z/Z_\odot\right)}$</div>



<div id="title">

# Carbon and Iron Deficiencies in Quiescent Galaxies at $z=1-3$ from _JWST_-SUSPENSE:\ Implications for the Formation Histories of Massive Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.02556-b31b1b.svg)](https://arxiv.org/abs/2407.02556)<mark>Appeared on: 2024-07-05</mark> -  _Submitted to ApJ; 18 pages, 6 figures, 1 table_

</div>
<div id="authors">

A. G. Beverage, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We present the stellar metallicities and multi-element abundances (C, Mg, Si, Ca, Ti, Cr, and Fe) of 15 massive (log $M/M_\odot=10.2-11.2$ ) quiescent galaxies at $z=1-3$ , derived from ultradeep _JWST_ -SUSPENSE spectra. Compared to quiescent galaxies at $z\sim0$ , these galaxies exhibit a deficiency of 0.25 dex in [ C/H ] , 0.16 dex in [ Fe/H ] , and 0.07 dex in [ Mg/H ] , implying rapid formation and quenching before significant enrichment from asymptotic giant branch stars and Type Ia supernovae. Additionally, we find that galaxies that form at higher redshift have higher [ Mg/Fe ] and lower [ Fe/H ] and [ Mg/H ] , irrespective of their observed redshift. The evolution in [ Fe/H ] and [ C/H ] is therefore primarily explained by lower redshift samples naturally including galaxies with longer star-formation timescales. On the other hand, the lower [ Mg/H ] can be explained by galaxies forming at earlier epochs expelling larger gas reservoirs during their quenching phase. Consequently, the mass-metallicity relation, primarily reflecting [ Mg/H ] , is also lower at $z=1-3$ compared to the lower redshift relation, though the slopes are similar. Finally, we compare our results to standard stellar population modeling approaches employing solar abundance patterns and non-parametric star-formation histories (using Prospector ). Our SSP-equivalent ages agree with the mass-weighted ages from Prospector , while the metallicities disagree significantly. Nonetheless, the metallicities better reflect [ Fe/H ] than total [ Z/H ] . We also find that star-formation timescales inferred from elemental abundances are significantly shorter than those from Prospector , and we discuss the resulting implications for the early formation of massive galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2407.02556/./fig4_abundances.png" alt="Fig3" width="100%"/>

**Figure 3. -** The formation time ($t_{\rm form}$), [Fe/H], [Mg/H], [Fe/Mg], and [C/Mg] as a function of velocity dispersion for the $z=1-$3_JWST_-SUSPENSE quiescent galaxy sample (circles) and for other various measurements at similar redshifts from the literature. Each point is colored by its spectroscopic redshifts. To guide the eye, we include the best-fit relations and corresponding confidence intervals in each panel, fit to the $z=1-3$ data point. In the $t_{\rm form}$ panel, we instead present two relations, one for $z<1.5$ and the other for $z>1.5$. For comparison, we also include the abundances of stacks of quiescent galaxies at $z\sim0$ from SDSS  (conroy_early-type_2014, beverage_carbon_2023)  and at $z\sim0.7$ from LEGA-C  (beverage_carbon_2023) . At constant $\sigma$, the SUSPENSE galaxies form earlier and are more deficient in Fe and C than the $z<0.7$ galaxies. (*fig:abundance_panels*)

</div>
<div id="div_fig2">

<img src="tmp_2407.02556/./formation_time_v_xh_cM.png" alt="Fig5" width="100%"/>

**Figure 5. -** [Fe/H], [Mg/H], and [Mg/Fe] as a function of galaxy formation time for massive quiescent galaxies at $z=0-3$. Data points for the redshift range $z=1-3$ are color-coded based on their spectroscopic redshift, and their shapes follow the same legend as Figure \ref{fig:abundance_panels}. The red circles [pink triangles] represent individual quiescent galaxies from SDSS [LEGA-C] (zhuang_glimpse_2023,  beverage_carbon_2023) . Typical uncertainties for the SDSS and LEGA-C points are shown in the bottom right of each panel. We remove the first-order dependence on stellar mass by adjusting all measurements to reflect the abundances at $M_*=10^{11}\;M_\odot$(refer to the text for details). We calculate the Pearson correlation coefficients, including all galaxies at all redshifts, and list the corresponding correlations (r) and $p$-values in the top left corner of each panel. Regardless of observed redshift, galaxies that form earlier have lower [Fe/H], slightly lower [Mg/H], and higher [Mg/Fe], consistent with more rapid formation and efficient quenching at earlier times. (*fig:formation_time-label*)

</div>
<div id="div_fig3">

<img src="tmp_2407.02556/./abundance_pattern_xfe_xh.png" alt="Fig4" width="100%"/>

**Figure 4. -** The abundance patterns of the $z=1-3$ quiescent galaxies (blue squares). We show the absolute abundances, [X/H], in the top panel and the abundance ratios, [X/Fe] in the bottom panel, with the solar values marked with a dashed black line. The thick solid blue line with square markers shows the median abundance pattern of the $z=1-3$ quiescent galaxies. For comparison, we include the average abundance patterns of $z\sim0$(red solid line with circle markers) and $z\sim0.7$(pink solid line with triangle markers) galaxies with the _same_ velocity dispersions ($175-275 \mathrm{km s^{-1}}$). The $z=1-3$ galaxies are significantly deficient in [C/H] and [Fe/H] compared to lower redshift samples. The SUSPENSE galaxies also have enhanced [$\alpha$/Fe](Mg, Si, Ca), which is primarily driven by deficient [Fe/H]. (*fig:abundance_pattern*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.02556"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Msun}{\ensuremath{\rm M_\odot}}$
$\newcommand{\mdot}{\ensuremath{\rm M_\odot yr^{-1}}}$
$\newcommand{\Lsun}{\ensuremath{\rm L_\odot}}$
$\newcommand{\Rsun}{\ensuremath{\rm R_\odot}}$
$\newcommand{\fuse}{\emph{FUSE}}$
$\newcommand{\stis}{\emph{STIS}}$
$\newcommand{\cosp}{\emph{COS}}$
$\newcommand{\hst}{\emph{HST}}$
$\newcommand{\eso}{\emph{ESO}}$
$\newcommand{\xshooter}{\emph{X-shooter}}$
$\newcommand{\teff}{T_\text{eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\loggc}{\log g_\text{c}}$
$\newcommand{\vturb}{\varv_{\rm turb}}$
$\newcommand{\vmac}{\varv_{\rm mac}}$
$\newcommand{\vsini}{\varv\sin i}$
$\newcommand{\vrad}{\varv_{\rm rad}}$
$\newcommand{\radius}{R_{*}/R_{\odot}}$
$\newcommand{\mass}{M_{*}/M_{\odot}}$
$\newcommand{\logL}{\log (L_{*}/L_{\odot})}$
$\newcommand{\rstar}{R_{*}}$
$\newcommand{\mstar}{M_{*}}$
$\newcommand{\zsun}{{\rm Z}_{\odot}}$
$\newcommand{\lsun}{{\rm L}_{\odot}}$
$\newcommand{\ebv}{\rm E(B-V)}$
$\newcommand{\Rv}{R_{\rm v}}$
$\newcommand{\Mv}{M_{\rm v}}$
$\newcommand{\cmfgen}{\textsc{CMFGEN}}$</div>



<div id="title">

# X-Shooting ULLYSES: Massive Stars at low metallicity 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.03137-b31b1b.svg)](https://arxiv.org/abs/2407.03137)<mark>Appeared on: 2024-07-05</mark> -  _18+15 pages, 21+4 figures, under review at A&A, condensed abstract_

</div>
<div id="authors">

A. Sander, et al. -- incl., <mark>C. Larkin</mark>

</div>
<div id="abstract">

**Abstract:** The spectral analysis of hot, massive stars is a fundamental astrophysical method to obtain their intrinsic properties and determine their feedback. With their inherent, radiation-driven winds, the quantitative spectroscopy for hot, massive stars requires a detailed numerical modeling of the atmosphere and an iterative treatment to obtain the best solution within a given physical and numerical framework. We present an overview of different techniques for the quantitative spectroscopy of hot stars employed within the X-Shooting ULLYSES collaboration, ranging from grid-based approaches to tailored spectral fits. By performing a blind test for selected targets, we gain an overview about the similarities and differences of the resulting stellar and wind parameters. Our study is not a systematic benchmark between different codes or methods, but aims to provide an overview of the parameter spread caused by different approaches. For three different stars from the XShooting ULLYSES sample (SMC O5 star AzV 377, LMC O7 star Sk -69 $^{\circ}$ 50, and LMC O9 star Sk -66 $^{\circ}$ 171), we employ different stellar atmosphere codes (CMFGEN, Fastwind , PoWR) and different strategies to determine their best-fitting model solutions. For our analyses, UV and optical spectroscopy are used to determined the stellar and wind properties with some methods relying purely on optical data for comparison. To determine the overall spectral energy distribution, we further employ additional available photometry from the literature. The effective temperatures for each of three different sample stars agree within $3 $ kK while the differences in $\log g$ can be up to $0.2 $ dex. Luminosity differences of up to $0.1 $ dex are resulting from different reddening assumptions, which seem to be systematically larger for the methods employing a genetic algorithm. All sample stars are found to be nitrogen-enriched. The terminal wind velocities are surprisingly similar and do not strictly follow the $\varv_\infty$ - $T_\text{eff}$ -relation. We find a reasonable agreement for the derived stellar and wind parameters between the different methods. Tailored fitting methods tend to be able to minimize or avoid discrepancies obtained with more course or automatized treatments. The inclusion of UV spectral data is essential for the determination of realistic wind parameters. For one target (Sk -69 $^{\circ}$ 50), we find clear indications of an evolved status.

</div>

<div id="div_fig1">

<img src="tmp_2407.03137/./Figures/uv2_AV377-crop.png" alt="Fig4" width="100%"/>

**Figure 4. -** Comparison between the main UV profiles for AzV 377. The panels, from (a) to (h) depicts respectively the profiles of $\ion${O}{vi} 1032/1038 Å,
 $\ion${S}{iv} 1063/1073Å (and $\ion${He}{ii} 1085 Å), $\ion${P}{v} 1118/1128 Å, $\ion${C}{iii} 1176 Å, $\ion${N}{v} 1238/1242 Å, $\ion${Si}{iv} 1394/1403 Å, $\ion${C}{iv} 1548/1551 Å, $\ion${N}{iv} 1718 Å. Interstellar Ly$\alpha$ and Ly$\beta$ absorption affects some of the diagnostics, most notably the wing of $\ion${N}{v} 1238/1242 Å in case of larger terminal velocities (e.g., seen in AzV 377). (*fig:uv-AV377*)

</div>
<div id="div_fig2">

<img src="tmp_2407.03137/./Figures/SED_AV377_zoom-crop.png" alt="Fig11" width="100%"/>

**Figure 11. -** Zoom-in comparison of the spectral energy distributions from the different models for AzV 377 around the applied photometry (small upper panels, crosses mark photometric measurements) and the flux-calibrated UV spectra (big lower panel). (*fig:SED-zoom-AV377*)

</div>
<div id="div_fig3">

<img src="tmp_2407.03137/./Figures/he-hydrogens_AV377-crop.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison between the different methods in the Balmer lines and $\ion${He}{ii} $\lambda$4541 region for AzV 377. (*fig:balmer-AV377*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.03137"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Rjup}{R_{\mathrm{J}}}$
$\newcommand{\Mjup}{M_{\mathrm{J}}}$
$\newcommand{\Rp}{R_{\mathrm{P}}}$
$\newcommand{\Mp}{M_{\mathrm{P}}}$
$\newcommand{\Rs}{R_{\mathrm{*}}}$
$\newcommand{\Ms}{R_{\mathrm{*}}}$
$\newcommand{\Teq}{T_{\mathrm{eq}}}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\obliquity}{|\lambda|}$</div>



<div id="title">

# BOWIE-ALIGN: A JWST comparative survey of aligned vs misaligned hot Jupiters to test the dependence of atmospheric composition on migration history

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.03198-b31b1b.svg)](https://arxiv.org/abs/2407.03198)<mark>Appeared on: 2024-07-05</mark> -  _13 pages, 8 figures, submitted to RASTI_

</div>
<div id="authors">

J. Kirk, et al. -- incl., <mark>E.-M. Ahrer</mark>

</div>
<div id="abstract">

**Abstract:** A primary objective of exoplanet atmosphere characterisation is to learn about planet formation and evolution, however, this is challenged by degeneracies. To determine whether differences in atmospheric composition can be reliably traced to differences in evolution, we are undertaking a new survey with JWST to compare the compositions of a sample of hot Jupiters that orbit F stars above the Kraft break with different orbital alignments. Under the assumption that aligned planets migrate through the inner disc, while misaligned planets migrate after disc dispersal, the act of migrating through the inner disc should lead to a measurable difference in the C/O between aligned and misaligned planets. We expect the amplitude and sign of this difference to depend on the amount of planetesimal accretion and whether silicates accreted from the inner disc release their oxygen. Here, we identify all known exoplanets that are suitable for testing this hypothesis, describe our JWST survey, and use noise simulations and atmospheric retrievals to estimate our survey's sensitivity. With the selected sample of four aligned and four misaligned hot Jupiters, we will be sensitive to the predicted differences in C/O between aligned and misaligned hot Jupiters for a wide range of model scenarios.

</div>

<div id="div_fig1">

<img src="tmp_2407.03198/./figures/Anna_models_with_measurement_uncertainties.png" alt="Fig4" width="100%"/>

**Figure 4. -** The C/O and [O/H](= Z) results of two models from the suite of models run in \citetalias{PenzlinBooth2024}\citeyear{PenzlinBooth2024}. Left panel: The C/O and [O/H] from the fiducial model (disc scenario 1), showing that aligned, disc-migrated planets (orange points) tend towards lower C/O than misaligned, high-eccentricity migrated planets (blue points) due to the accretion of O-rich ices in the inner disc. The dashed and dotted lines indicate the two values of metallicity and four values of C/O that were adopted in our simulation setup here. The white circles with uncertainties show the precision with which we'll be able to measure these values from an eight-planet sample (four aligned and four misaligned planets). Right panel: The C/O and [O/H] from the silicate rainout model (disc scenario 2), whereby silicates do not release their O into the atmospheres of exoplanets upon accretion. In this case, aligned, disc-migrated planets (orange points) tend towards higher C/O than misaligned, high-eccentricity migrated planets (blue points). Again, the dashed and dotted lines indicate the two values of metallicity and four values of C/O that were adopted in our simulation setup here. (*fig:anna_models*)

</div>
<div id="div_fig2">

<img src="tmp_2407.03198/./figures/posteriors_fiducial_extra_fit_params.png" alt="Fig5" width="100%"/>

**Figure 5. -** The retrieved $\log Z$ and C/O posteriors from the model atmospheres with $\log Z$ and C/O set by the fiducial disc model (scenario 1). The left column corresponds to the $3\times$ solar metallicity cases, and the right column to $30\times$ solar. Top row: the combined $\log Z$ posteriors from the aligned planets for a *total* sample size of 4 (2 aligned, 2 misaligned), 8 (4 aligned, 4 misaligned) and 16 (7 aligned, 9 misaligned) planets, with the legend giving the standard deviations of these distributions. The darkest line corresponds to our actual sample size of 8 planets. The vertical dashed and dotted orange lines indicate the mean and standard deviation of the 8 planet posterior. The vertical dotted black line indicates the input value. Second row: $\log Z$ posteriors for the misaligned planets. Bottom row: the C/O posteriors for both the aligned (orange) and misaligned planets (blue). (*fig:posteriors_fiducial*)

</div>
<div id="div_fig3">

<img src="tmp_2407.03198/./figures/opacity_contributions_TrES-4b.png" alt="Fig2" width="100%"/>

**Figure 2. -** Two example forward models used in this analysis, both created for TrES-4b (an aligned target). Top panel: a $3\times$ solar metallicity, $\mathrm{C/O} = 0.28$ forward model (black dashed line), corresponding to the fiducial disc model (disc scenario 1, Section \ref{sec:model_predictions} and Figure \ref{fig:anna_models}, left panel). The grey uncertainties correspond to the estimated JWST precision at a spectral resolution of $R=100$. The colour lines show the contribution of different species to the overall opacity, using line lists for $H_2$O from \protect[Polyansky, et. al (2018)](https://ui.adsabs.harvard.edu/abs/2018MNRAS.480.2597P), $CO_2$ from \protect[Tashkun and Perevalov (2011)](https://ui.adsabs.harvard.edu/abs/2011JQSRT.112.1403T), and CO from \protect[Faure, et. al (2013)](https://ui.adsabs.harvard.edu/abs/2013JQSRT.116...79F), [ and Gordon (2017)](https://ui.adsabs.harvard.edu/abs/2017JQSRT.203....3G). Bottom panel: a $30\times$ solar metallicity, $\mathrm{C/O} = 0.80$ forward model (black line), corresponding to the silicate rainout disc model (disc scenario 2, Section \ref{sec:model_predictions} and Figure \ref{fig:anna_models}, right panel). (*fig:opacities*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.03198"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\prt}[2]{\frac{\partial #1}{\partial #2}}$
$\newcommand{\Prt}[1]{\frac{\partial}{\partial #1}}$
$\newcommand{\PPrt}[1]{\frac{\partial^2}{\partial #1^2}}$
$\newcommand{\todo}[1]{\textcolor{red}{\bf (todo: #1)}}$
$\newcommand{\checks}[1]{\textcolor{blue}{\bf #1}}$
$\newcommand{\new}[1]{\textcolor{black}{\bf #1}}$
$\newcommand{\quest}[1]{\textcolor{magenta}{\bf #1}}$
$\newcommand{\anna}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\rab}[1]{\textcolor{darkgreen}{#1}}$
$\newcommand{\jeo}[1]{\textcolor{blue}{#1}}$
$\newcommand{\kirk}[1]{\textcolor{orange}{#1}}$
$\newcommand{\ep}{e_\mathrm{p}}$
$\newcommand{\ap}{a_\mathrm{p}}$
$\newcommand{\St}{\mathrm{St}}$
$\newcommand{\rh}{r_\mathrm{hill}}$
$\newcommand{\Rsun}{\ensuremath{R_{\odot}}}$
$\newcommand{\Rjup}{\ensuremath{R_{\mathrm{J}}}}$
$\newcommand{\Mjup}{\ensuremath{M_{\mathrm{J}}}}$
$\newcommand{\Rp}{\ensuremath{R_{\mathrm{P}}}}$
$\newcommand{\Mp}{\ensuremath{M_{\mathrm{P}}}}$
$\newcommand{\Rs}{\ensuremath{R_{\mathrm{*}}}}$
$\newcommand{\Ms}{\ensuremath{R_{\mathrm{*}}}}$
$\newcommand{\Teq}{\ensuremath{T_{\mathrm{eq}}}}$
$\newcommand{◦ee}{\ensuremath{^{\circ}}}$
$\newcommand{\obliquity}{\ensuremath{|\lambda|}}$
$\newcommand{\um}{\ensuremath{\mu \text{m}}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\title[BOWIE-ALIGN: Modelling the compositions of disc- and high-e migrated gas giants]{BOWIE-ALIGN: How formation and migration histories of giant planets impact atmospheric compositions}$
$\author[Penzlin, Booth et al.]{Anna~B.T. Penzlin^{1}\thanks{E-mail: a.penzlin@imperial.ac.uk}\thanks{Authors contributed equally.},$
$Richard~A. Booth^{2}\footnotemark[2],$
$James Kirk^{1},$
$James E. Owen^{1},$
$E. Ahrer^{3},$
$\newauthor$
$Duncan A. Christie^{3},$
$Alastair B. Claringbold^{4,5},$
$Emma Esparza-Borges^{6,7},$
$M. López-Morales^{8},$
$\newauthor$
$N. J. Mayne^{9},$
$Mason McCormack^{10},$
$Annabella Meech^{8,11},$
$Vatsal Panwar^{4,5},$
$Diana Powell^{10},$
$\newauthor$
$Denis E. Sergeev^{9},$
$Jake Taylor^{11},$
$Peter J. Wheatley^{4,5},$
$Maria Zamyatina^{9},$
$BOWIE+ collaboration$
$\ ^{1}Astrophysics Group, Department of Physics, Imperial College London, Prince Consort Rd, London, SW7 2AZ, UK \ ^{2}School of Physics and Astronomy, University of Leeds, Leeds LS2 9JT, UK\ ^{3}Max Planck Institute for Astronomy (MPIA), Königstuhl 17, 69117 Heidelberg, Germany\ ^{4}Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK\ ^{5}Department of Physics, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK\ ^{6}Instituto de Astrofísica de Canarias, San Cristóbal de La Laguna, Tenerife E-38200, Spain \ ^{7}Departamento de Astrofísica, Universidad de La Laguna, San Cristóbal de La Laguna, Tenerife E-38200, Spain\ ^{8}Center for Astrophysics {\rm \mid} Harvard {\rm \&} Smithsonian, 60 Garden St, Cambridge, MA 02138, USA\ ^{9}Department of Physics and Astronomy, Faculty of Environment, Science and Economy, University of Exeter, Exeter EX4 4QL, UK\ ^{10}Department of Astronomy and Astrophysics, University of Chicago, IL, 60657, USA\ ^{11}Department of Physics, University of Oxford, Keble Road, Oxford, OX1 3RH, UK\ }$
$\date{Accepted XXX. Received YYY; in original form ZZZ}$
$\pubyear{2024}$
$\begin{document}$
$\label{firstpage}$
$\pagerange{\pageref{firstpage}--\pageref{lastpage}}$
$\maketitle$
$\begin{abstract}{$
$Hot Jupiters present a unique opportunity for measuring how planet formation history shapes present-day atmospheric composition. However, due to the myriad pathways influencing composition, a well-constructed sample of planets is needed to determine whether formation history can be accurately traced back from atmospheric composition.$
$To this end, the BOWIE-ALIGN survey will compare the compositions of 8 hot Jupiters around F stars, 4 with orbits aligned with the stellar rotation axis and 4 misaligned.$
$Using the alignment as an indicator for planets that underwent disc migration or high-eccentricity migration, one can determine whether migration history produces notable differences in composition between the two samples of planets.$
$This paper describes the planet formation model that motivates our observing programme. Our model traces the accretion of chemical components from the gas and dust in the disc over a broad parameter space to create a full, unbiased model sample from which we can estimate the range of final atmospheric compositions.$
$For high metallicity atmospheres (O/H\geq10 \times solar), the C/O ratios of aligned and misaligned planets diverge, with aligned planets having lower C/O (<0.25) due to the accretion of oxygen-rich silicates from the inner disc. However, silicates may$
$rain out instead of releasing their oxygen into the atmosphere. This would significantly increase the C/O of aligned planets (C/O >0.6), inverting the trend between the aligned and misaligned planets. Nevertheless, by comparing statistically significant samples of aligned and misaligned planets, we expect atmospheric composition to constrain how planets form.$
$}$
$\end{abstract}$
$\begin{keywords}$
$Planet formation -- Protoplanetary discs$
$\end{keywords}$
$\n\end{document}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\vec}[1]{\bm{\mathrm{#1}}}$</div>



<div id="title">

# BOWIE-ALIGN: How formation and migration histories of giant planets impact atmospheric compositions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.03199-b31b1b.svg)](https://arxiv.org/abs/2407.03199)<mark>Appeared on: 2024-07-05</mark> -  _11pages 10 figures, (appendix: 6 page, 4 figures), submitted to mnras_

</div>
<div id="authors">

A. B. Penzlin, et al. -- incl., <mark>E.-M. Ahrer</mark>

</div>
<div id="abstract">

**Abstract:** Hot Jupiters present a unique opportunity for measuring how planet formation history shapes present-day atmospheric composition. However, due to the myriad pathways influencing composition, a well-constructed sample of planets is needed to determine whether formation history can be accurately traced back from atmospheric composition.To this end, the BOWIE-ALIGN survey will compare the compositions of 8 hot Jupiters around F stars, 4 with orbits aligned with the stellar rotation axis and 4 misaligned.Using the alignment as an indicator for planets that underwent disc migration or high-eccentricity migration, one can determine whether migration history produces notable differences in composition between the two samples of planets.This paper describes the planet formation model that motivates our observing programme. Our model traces the accretion of chemical components from the gas and dust in the disc over a broad parameter space to create a full, unbiased model sample from which we can estimate the range of final atmospheric compositions.For high metallicity atmospheres ( $O/H\geq10 \times$ solar), the C/O ratios of aligned and misaligned planets diverge, with aligned planets having lower C/O ( $<0.25$ ) due to the accretion of oxygen-rich silicates from the inner disc. However, silicates mayrain out instead of releasing their oxygen into the atmosphere. This would significantly increase the C/O of aligned planets (C/O $>0.6$ ), inverting the trend between the aligned and misaligned planets. Nevertheless, by comparing statistically significant samples of aligned and misaligned planets, we expect atmospheric composition to constrain how planets form.

</div>

<div id="div_fig1">

<img src="tmp_2407.03199/./fig/disc.png" alt="Fig1" width="100%"/>

**Figure 1. -** Molecule abundances throughout the disc in gas and ice (top) and the resulting C/O ratio (bottom). The left panels show a disc with nearly no dust drift. The right panels show the enhancements near the icelines of fast drifting dust. To achieve a constant flux in all discs, the level of solid composition to gas composition scales with the drift velocity. (*fig:disc*)

</div>
<div id="div_fig2">

<img src="tmp_2407.03199/./fig/image10.png" alt="Fig3" width="100%"/>

**Figure 3. -** The final atmosphere compositions from the full parameter range in Table \ref{tab:models}. The marker symbol corresponds to the temperature ($ \times \rightarrow 125$ K, $ + \rightarrow 150$ K, $\circ \rightarrow 200$ K). Orange markers represent the disc migrated planets and blue markers represent the high-eccentricity migrated planets. Saturated markers show gas dominated accretion. The three different tracks correspond to different dust-to-gas flux ratios and are labelled. The dashed grey line indicates the critical values for models with planetesimal accretion fraction less than $<10^{-4}$, all such models are confined to the left side of the line. The coloured dashed line indicates the minimum C/O ratio for gas dominated accretion ($\mathrm{O_{gas}}/\mathrm{O_{dust}}>1$). (*fig:all_set*)

</div>
<div id="div_fig3">

<img src="tmp_2407.03199/./fig/O2O_fin1.png" alt="Fig9" width="100%"/>

**Figure 9. -** The final atmosphere compositions when counting all, only the final 10 \% and final 1 \% of accreted material. The marker symbols and colours are the same as in Fig. \ref{fig:all_set}. (*fig:fin_set*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.03199"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

114

  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
